## Choosing multiple tools

https://python.langchain.com/docs/use_cases/tool_use/multiple_tools/

In [6]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
import getpass
import os
from openai._base_client import SyncHttpxClientWrapper, AsyncHttpxClientWrapper


# os.environ["OPENAI_API_KEY"] = "{your_key}"
# # os.environ["OPENAI_API_KEY"] = getpass.getpass()


# llm = ChatOpenAI(
#     model="gpt-3.5-turbo-0125",
#     temperature=0
# )
http_client = SyncHttpxClientWrapper(verify=False)

llm = ChatOpenAI(
    model_name="qwen32b-distill-r1",
    openai_api_key="EMPTY",
    openai_api_base="http://10.4.119.108:8000/v1",
    # http_client=http_client
)

llm.invoke("hello")

In [7]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [8]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [9]:
# 增加额外的返回是没用的
chain.invoke("What's 2 times 5?, please add some description to the answer.")


BadRequestError: Error code: 400 - {'object': 'error', 'message': '"auto" tool choice requires --enable-auto-tool-choice and --tool-call-parser to be set', 'type': 'BadRequestError', 'param': None, 'code': 400}

In [31]:
# 复杂的运算会出错，应该是一次选择了工具，而非逐步计算，这个估计得 Agent 实现
chain.invoke("What's 2 times 4 then times 5 then times 8?")

[{'name': 'multiply',
  'args': {'first_int': 2, 'second_int': 4},
  'id': 'call_tUnvfSiMCIfCZ1XUysNknneG',
  'output': 8},
 {'name': 'multiply',
  'args': {'first_int': 8, 'second_int': 5},
  'id': 'call_u6bNMGxodQ0FAfJCk544chdc',
  'output': 40}]

In [35]:
# 中文也行
chain.invoke("3的3次方是多少")

[{'name': 'exponentiate',
  'args': {'base': 3, 'exponent': 3},
  'id': 'call_RmOaZTX0FYIxl4U4WdqT5tgR',
  'output': 27}]